<img src="./images/DLI_Header.png" style="width: 400px;">

# 部署模型
现在我们已对模型进行了有效的训练，接下来就该实际应用它了。本练习中，我们将给模型展示新图像，让它判断新图像里的手语所对应的字母表中的正确字母。

## 目标
完成这一章节后，您将能够：
* 从磁盘加载一个已经训练过的模型；
* 为基于不同格式图像训练而得到的模型，重新格式化图像
* 使用训练过的模型从未见过的新图像进行推理，并评估其性能

## 加载模型
我们当前正在使用新的 notebook，所以需要重新加载已保存的训练好的模型。前一个练习中的保存操作创建了一个名为`asl_model`的文件。我们可以通过选择同一文件名来加载该模型。

In [ ]:
from tensorflow import keras

model = keras.models.load_model('asl_model')

如要确保所有内容看起来全都完好无损，您可以再次查看模型的摘要。

In [ ]:
model.summary()

## 为模型准备图像
现在，我们就来使用模型对其先前从未见过的新图像进行预测，此过程也称为推理。我们在 `asl_images` 文件夹中为您提供了一组图像。请尝试在左侧的文件导航区域打开并浏览这些图像。

您会发现，此文件夹中的图像比数据集里的图像的分辨率要高得多，而且还是彩色的。别忘了，数据集中的图像为 28x28 像素，且为灰度图。请务必牢记，无论何时使用模型进行预测，输入皆须与训练模型所用数据的形状相匹配。对于此模型，训练数据集的形状如下：(27455, 28, 28, 1)，这对应于 27455 个 28x28 像素的图像，其中每个图像皆具有一个颜色通道（灰度图）。

### 显示图像
使用模型对新图像进行预测时，同时显示输入图像对我们也有用。我们可以使用 matplotlib 库来执行此操作。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

In [ ]:
show_image('asl_images/b.png')

### 缩放图像
数据集中的图像为 28x28 像素，且为灰度图。我们需要确保将相同大小和颜色的图像传入模型的`predict`方法中。目前有好几种方法使用 Python 对图像进行编辑，但 Keras 已有一个十分有效的内置实用程序。

In [ ]:
from tensorflow.keras.preprocessing import image as image_utils

def load_and_scale_image(image_path):
    image = image_utils.load_img(image_path, color_mode="grayscale", target_size=(28,28))
    return image

In [ ]:
image = load_and_scale_image('asl_images/b.png')
plt.imshow(image, cmap='gray')

### 准备图像以供预测

我们已经有了一张 28x28 像素的灰度图，就要准备好将其传入模型进行预测了。但首先，我们还需改变它的形状，使之与训练模型所采用的数据集的形状相匹配。改变形状前，我们需要将图像转换为更原始的格式。我们将通过一个名为 `image_to_array` 的 Keras 实用程序完成此操作。

In [ ]:
image = image_utils.img_to_array(image)

现在我们可以改变图像的形状，为预测做好准备。

In [ ]:
# This reshape corresponds to 1 image of 28x28 pixels with one color channel
image = image.reshape(1,28,28,1) 

最后，请务必牢记要将数据归一化（使所有像素值介于 0 至 1 之间），就像我们对训练数据集的处理操作一样：

In [ ]:
image = image / 255

## 进行预测

现在我们已准备好进行预测！我们可以将预处理的图像传入模型的`predict`方法来完成此操作。

In [ ]:
prediction = model.predict(image)
print(prediction)

### 理解预测结果

预测结果的格式是一个长度为 24 的一维数组。尽管看起来略有差异，但此格式与 y_train 和 y_test 中的“二值化”的多分类数组相同。数组中的每个元素均为 0 至 1 之间的概率，代表了每个类别的置信度。为了使这个结果更具可读性，我们可以先找出该数组中具有最高概率的那个元素。使用 `numpy` 库和 [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 函数就能轻松完成此操作。

In [ ]:
import numpy as np
np.argmax(prediction)

预测结果的数组中的每个元素代表手语字母表中的一个可能的字母。请记住，j 和 z 已排除在外，因为这两个字母涉及手部动作，而我们仅需要处理静态照片。让我们在预测结果的数组的索引值和相应字母之间创建一个映射。

In [ ]:
# Alphabet does not contain j or z because they require movement
alphabet = "abcdefghiklmnopqrstuvwxy"
dictionary = {}
for i in range(24):
    dictionary[i] = alphabet[i]
dictionary

现在我们根据输入预测的索引值找到对应的字母。

In [ ]:
dictionary[np.argmax(prediction)]

## 练习：汇总一切

让我们将上述与预测相关的内容全部置入一个函数中，这样我们就能对输入的图像进行预测。利用上面使用过的功能和步骤，在下方实现这一函数。如果需要帮助，可单击下方的三个点来显示答案。

In [ ]:
def predict_letter(file_path):
    # Show image
    FIXME
    # Load and scale image
    image = FIXME
    # Convert to array
    image = FIXME
    # Reshape image
    image = FIXME
    # Normalize image
    image = FIXME
    # Make prediction
    prediction = FIXME
    # Convert prediction to letter
    predicted_letter = FIXME
    # Return prediction
    return predicted_letter   

## 答案

单击下方的三个点来显示答案。

```python
# Answer
def predict_letter(file_path):
    show_image(file_path)
    image = load_and_scale_image(file_path)
    image = image_utils.img_to_array(image)
    image = image.reshape(1,28,28,1) 
    image = image/255
    prediction = model.predict(image)
    # convert prediction to letter
    predicted_letter = dictionary[np.argmax(prediction)]
    return predicted_letter
```

In [ ]:
predict_letter("asl_images/b.png")

让我们再用 asl_images 数据集中带有'a'的字母执行预测函数：

In [ ]:
predict_letter("asl_images/a.png")

## 总结
您已在这些练习中出色地完成了工作！您已经从头开始完成了高精度模型训练的整个流程，然后采用该模型进行了有价值的新预测。如果有时间，您可上传自己的图像放到 asl_data 文件夹中，并用这些图像测试模型。这也是学习手语的好机会！例如，您可以尝试用手语比出您名字中的字母。

您可以想象一下，如何在应用中使用此模型来教人手语，甚至帮助失声者与计算机交互。如果您擅长 Web 开发，您甚至可以通过名为 [TensorFlow.js](https://www.tensorflow.org/js) 的库在浏览器中使用这个模型。

### 清理显存
继续进行后面的内容前，请执行以下单元清理 GPU 显存。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

我们希望你喜欢这些练习。在下一章节中，您将学习在没有强大的数据集时，如何利用深度学习。如果你想阅读和了解当下热门的推理技术，可以查看 [这篇很棒的论文](https://research.fb.com/wp-content/uploads/2018/12/Machine-Learning-at-Facebook-Understanding-Inference-at-the-Edge.pdf)。

现在您已熟悉构建模型并对其工作原理有了一定的了解，我们将把精力转向使用预训练模型来加速您的工作的强大技术。请继续学习下一个章节：[*预训练模型*](./05a_doggy_door.ipynb)。